In [ ]:
import json
from datetime import datetime, time

from pymongo.mongo_client import MongoClient
import bson


* A partir du fichier trips_.json, remplacez les espaces des clés par un underscore, puis mettez en base en changeant les chaîne de caractères qui concerne des grandeurs numériques par le type integer ou float et les champs de date ($date) doit être sous format datetime.datetime. Ainsi les champs préfixé '$number...' suivent  l'exemple ci-dessous(cherchez du côté de la librairie bson, il existe une fonction utilitaire, il faut aussi réduire la profondeur):

```json
{'tripduration': {'$numberInt': "379"}}
```

en :

```json
{'tripduration': 379}
```



In [21]:
# On parse le fichie BSON avec la fonction bson.json_util.loads()
# Elle transformera les indices de type en type utilisable. 
# $numberInt -> int $date -> datetime, etc... 
# on peut les faire réapparaitre avec bson.json_util.dumps()
parsed_lines = []
with open("trips_.json", 'r') as file:
    for line in file:
        parsed_lines.append(bson.json_util.loads(line))

for i in range(3):
    print(parsed_lines[i])
    # print(bson.json_util.dumps(parsed_lines[i], json_options=bson.json_util.CANONICAL_JSON_OPTIONS))

{'tripduration': 307, 'start station id': 3118, 'start station name': 'McGuinness Blvd & Eagle St', 'end station id': 3119, 'end station name': 'Vernon Blvd & 50 Ave', 'bikeid': 23477, 'usertype': 'Subscriber', 'birth year': 1987, 'gender': 1, 'start station location': {'type': 'Point', 'coordinates': [-73.95284, 40.73555]}, 'end station location': {'type': 'Point', 'coordinates': [-73.95411749, 40.74232744]}, 'start time': datetime.datetime(2016, 1, 1, 7, 4, 31), 'stop time': datetime.datetime(2016, 1, 1, 7, 9, 38)}
{'tripduration': 3234, 'start station id': 500, 'start station name': 'Broadway & W 51 St', 'end station id': 337, 'end station name': 'Old Slip & Front St', 'bikeid': 20545, 'usertype': 'Customer', 'birth year': '', 'gender': 0, 'start station location': {'type': 'Point', 'coordinates': [-73.98336183, 40.76228826]}, 'end station location': {'type': 'Point', 'coordinates': [-74.00838676, 40.7037992]}, 'start time': datetime.datetime(2016, 1, 2, 8, 17, 50), 'stop time': dat

In [23]:
# on peut maintenant les insérer dans la base donnée. On fera le nettoyage des attributs après

connection = MongoClient() 
db = connection["cas_pratique"]
collection = db["trips"]

collection.insert_many(parsed_lines)

InsertManyResult([ObjectId('688a2ddab7998daf5c006af5'), ObjectId('688a2ddab7998daf5c006af6'), ObjectId('688a2ddab7998daf5c006af7'), ObjectId('688a2ddab7998daf5c006af8'), ObjectId('688a2ddab7998daf5c006af9'), ObjectId('688a2ddab7998daf5c006afa'), ObjectId('688a2ddab7998daf5c006afb'), ObjectId('688a2ddab7998daf5c006afc'), ObjectId('688a2ddab7998daf5c006afd'), ObjectId('688a2ddab7998daf5c006afe'), ObjectId('688a2ddab7998daf5c006aff'), ObjectId('688a2ddab7998daf5c006b00'), ObjectId('688a2ddab7998daf5c006b01'), ObjectId('688a2ddab7998daf5c006b02'), ObjectId('688a2ddab7998daf5c006b03'), ObjectId('688a2ddab7998daf5c006b04'), ObjectId('688a2ddab7998daf5c006b05'), ObjectId('688a2ddab7998daf5c006b06'), ObjectId('688a2ddab7998daf5c006b07'), ObjectId('688a2ddab7998daf5c006b08'), ObjectId('688a2ddab7998daf5c006b09'), ObjectId('688a2ddab7998daf5c006b0a'), ObjectId('688a2ddab7998daf5c006b0b'), ObjectId('688a2ddab7998daf5c006b0c'), ObjectId('688a2ddab7998daf5c006b0d'), ObjectId('688a2ddab7998daf5c006b

In [ ]:
# Nettoyage des noms des clés. 

res = collection.aggregate([
    {"$project": {"items": { "$objectToArray": "$item" }}}
])

res

OperationFailure: Invalid $project :: caused by :: FieldPath field names may not start with '$'. Consider using $getField or $setField., full error: {'ok': 0.0, 'errmsg': "Invalid $project :: caused by :: FieldPath field names may not start with '$'. Consider using $getField or $setField.", 'code': 16410, 'codeName': 'Location16410'}

Après avoir mis en base et uniquement, assurez-vous qu'il n'y a pas de doublons, si il y en a, écrivez une fonction python permettant de les retirer (sans à avoir à faire de téléchargement)

De plus, des hypothèses d'incohérences ont été émises par les parties prenantes, il s'agit d'écrire pour chacune de ces hypothèses une fonction python qui va détecter les documents incriminés et les flagger avec un champs supplémentaire pour rectification/enquête ultérieure :

* Un vélo loué deux fois, mais la deuxième période de location démarre avant le rendu de la première période de location, ce qui n'est pas normal, ou tout autre chevanchement de période.
* Un utilisateur trop jeune (le service est réservé au + de 13 ans)
* La date de naissance n'est pas renseignée
* Des locations trop courtes (1 secondes)
* Des temps de location incohérents par rapport au start_time et au end_time


Faites un rapport détaillé des anomalies trouvées (nombres d'occurences)

Puis, après avoir fait un nettoyage et écarté les locations suspectes, répondez aux questions métier qui suivent :

* Quels sont les 5 trajets (start station → end station) les plus fréquents pour les utilisateurs de genre féminin ? (gender = 1)

* Quel est le nombre total de trajets par type d’utilisateur (Subscriber vs Customer) pour l’année 2023 ?

* Quelle est la durée moyenne des trajets par station de départ pour les trajets commençant entre 7h et 9h ?

* Quel est le top 3 des stations avec la plus forte augmentation du nombre de trajets entre deux années consécutives ?

* Pour chaque catégorie d’âge, quelle est la durée médiane des trajets ?

* Quels sont les utilisateurs les plus actifs (10%) et leur contribution au nombre total de trajets ?

* Quelle est la répartition des trajets par jour de la semaine (lundi, mardi, etc.) ?

* Quel est le temps moyen passé en trajet pour chaque genre, filtré sur les trajets de plus de 10 minutes ?

* Combien de trajets ont démarré dans une station donnée pendant les heures de pointe (ex. 7h-9h et 17h-19h) ?

* Quelles sont les 5 stations avec la plus grande diversité d’utilisateurs (nombre distinct d’user id) ?